In [1]:
import numpy as np
import pandas as pd
file_ap = 'applications.csv'
app_df = pd.read_csv(file_ap)
app_df.head()

,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id
0,11.30.2022 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3
1,11.30.2022 10:26:39,NaN,36,Чоловік,Public services / Government,Single,3.0,"Вища (бакалавр, спеціаліст, магістр)",NaN,63dfcf8e6904186650d6814279fbe42f
2,11.30.2022 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498
3,11.30.2022 10:27:31,1500.0,23,Жінка,Telecom,Single,0.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,5847ac62cc9eac5e323c2517dcc91ad1
4,11.30.2022 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3


### Cleaning applications.csv

In [2]:
app_df.drop_duplicates(inplace=True)
app_df['External Rating'].fillna(value=0,inplace=True)
app_df['Education level'].fillna(value='Average',inplace=True)
app_df.head(10)

,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id
0,11.30.2022 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3
1,11.30.2022 10:26:39,NaN,36,Чоловік,Public services / Government,Single,3.0,"Вища (бакалавр, спеціаліст, магістр)",NaN,63dfcf8e6904186650d6814279fbe42f
2,11.30.2022 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498
3,11.30.2022 10:27:31,1500.0,23,Жінка,Telecom,Single,0.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,5847ac62cc9eac5e323c2517dcc91ad1
4,11.30.2022 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3
5,11.30.2022 10:27:38,16500.0,31,Чоловік,E-commerce,Single,8.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,f720bf9c5c4c3e10a8568c1699847696
6,11.30.2022 10:27:42,4200.0,30,Чоловік,Media,Married,1.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,0aaf59fb3ef90f50ccd4800312e5c271
7,11.30.2022 10:27:44,3600.0,24,Чоловік,E-commerce,Married,1.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,aa54d6e672e4233febadbafdfd048327
8,11.30.2022 10:28:00,1950.0,27,Чоловік,Automotive,Married,1.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,2eed2883cd1673b21b2ce89d1115c245
9,11.30.2022 10:28:00,NaN,17,Чоловік,Інша,Married,2.0,Ще студент вишу,NaN,9c7a640dccb37d9f2164bac1deb0edfa


In [3]:
file_ind = 'industries.csv'
ind_df = pd.read_csv(file_ind)
ind_df.head()

,Industry,Score
0,Blockchain,0
1,Public services / Government,20
2,Adtech / Advertising,10
3,Telecom,15
4,Automotive,15


### Calculate the application score according to the guidelines:
The score should be a number from 0 to 100.
The score is the sum of points awarded to the application based on 6 criteria.
The score is zero if there is no 'Amount' value or if 'External Rating' is zero.
Components of the score:

If the candidate's age is between 35 and 55 years old, add 20 points to the score.
If the application was submitted during the week, excluding the weekend, add 20 points to the score.
If the candidate is married, add 20 points to the score.
If the candidate lives in Warsaw or the Masovian Voivodeship, add 10 points to the score.
The 'Score' from the industries.csv table is also added to the score (ranging from 0 to 20 points).
If the 'External Rating' value is greater than or equal to 7, add 20 points to the score.
If the 'External Rating' value is less than or equal to 2, subtract 20 points from the score.

In [6]:
merged = pd.merge(app_df, ind_df, on='Industry', how='left')

merged['Applied at'] = pd.to_datetime(merged['Applied at'], format="%m.%d.%Y %H:%M:%S", errors='coerce')


def set_rating(row):
    rating = 0
    if row['Amount'] == 0 or pd.isnull(row['Amount']) or row['External Rating'] == 0:
        return rating
    
    if 35 <= row['Age'] < 55:  
        rating += 20
        
    if row['Applied at'].weekday() in [5, 6]:  
        rating += 20
        
    if row['Marital status'] == 'Married':
        rating += 20
        
    if row['Location'] in ['Kyiv', 'Kyiv region']:
        rating += 10
        
    rating += row['Score']
    
    if row['External Rating'] >= 7:
        rating += 20
        
    if row['External Rating'] <= 2:
        rating -= 20
          
    return rating


merged['rating'] = merged.apply(set_rating, axis=1)






In [7]:
merged.head()

,Applied at,Amount,Age,Gender,Industry,Marital status,External Rating,Education level,Location,applicant_id,Score,rating
0,2022-11-30 10:26:37,12000.0,29,Чоловік,Blockchain,Other,8.0,"Вища (бакалавр, спеціаліст, магістр)",Івано-Франківськ чи область,99e7b0dc6cc05dd334d8f38dc26ce9b3,0,20
1,2022-11-30 10:26:39,NaN,36,Чоловік,Public services / Government,Single,3.0,"Вища (бакалавр, спеціаліст, магістр)",NaN,63dfcf8e6904186650d6814279fbe42f,20,0
2,2022-11-30 10:26:58,7500.0,34,Чоловік,Adtech / Advertising,Single,4.0,"Вища (бакалавр, спеціаліст, магістр)",Київ чи область,10dbafaeb46c09e96b6987c03bbb3498,10,10
3,2022-11-30 10:27:31,1500.0,23,Жінка,Telecom,Single,0.0,"Вища (бакалавр, спеціаліст, магістр)",Львів чи область,5847ac62cc9eac5e323c2517dcc91ad1,15,0
4,2022-11-30 10:27:34,8400.0,33,Жінка,Automotive,Single,6.0,"Вища (бакалавр, спеціаліст, магістр)",Житомир чи область,5d21f3795b50de8e8f8f8d5f48b754f3,15,15
